In [1]:
import pandas as pd
import numpy as np
import transformers
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

## Loading model and files

In [2]:
save_directory = './saved_BERT'

# Load DistilBERT model and tokenizer
tokenizer_BERT = transformers.DistilBertTokenizer.from_pretrained(save_directory)
model_BERT = transformers.DistilBertForSequenceClassification.from_pretrained(save_directory)

In [3]:
test_df = pd.read_csv('./inferencing/data/BERT.csv')

# Create test_texts and test_labels as lists

test_texts = test_df['text'].tolist()
test_labels = test_df['label'].tolist()

# Applying synonym replacement

In [4]:
import nltk
from nltk.corpus import wordnet
from transformers import BertTokenizer

nltk.download('wordnet')

def replace_with_synonym(text, synonym_ratio=0.10):
    tokens = tokenizer_BERT.tokenize(text)
    words = nltk.word_tokenize(text)
    
    num_words = len(words)
    num_synonyms = int(num_words * synonym_ratio)
    
    indices = np.random.choice(num_words, size=num_synonyms, replace=False)
    
    for idx in indices:
        word = words[idx]
        synonyms = set()
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
        if len(synonyms) > 0:
            synonym = next(iter(synonyms))
            tokens[idx] = synonym if tokens[idx] not in ['[CLS]', '[SEP]'] else tokens[idx]
    
    return tokenizer_BERT.convert_tokens_to_string(tokens)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yeetusonthefetus/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Applying masking

In [5]:
def apply_mask(text, mask_token='[MASK]', mask_ratio=0.60, words_to_mask=None):
    if words_to_mask is None:
        words_to_mask = ['depression', 'anxiety', 'suicidewatch', 'adhd', 'bpd', 'lonely',
                         'autism', 'schizophrenia', 'ptsd', 'addiction', 'alcoholism',
                         'depress', 'suicide', 'bipolar', 'addict', 'alchohol']
    
    words = text.split()
    words_lower = [word.lower() for word in words]
    mask_indices = []
    
    for idx, word in enumerate(words_lower):
        if word in words_to_mask and np.random.rand() < mask_ratio:
            mask_indices.append(idx)
    
    for idx in mask_indices:
        words[idx] = mask_token
    
    return ' '.join(words)

In [6]:
test_texts_syn = []

for t in test_texts:
    test_texts_syn.append(replace_with_synonym(t))

In [7]:
test_texts_mask = []

for t in test_texts:
    test_texts_mask.append(apply_mask(t))

In [8]:
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained(save_directory)

model_fine_tuned = DistilBertForSequenceClassification.from_pretrained(save_directory)



In [9]:
def compute_accuracy(test_labels, test='base' ):
    pred = []
    if test == 'base':
        for test_text in test_texts:
            predict_input = tokenizer_BERT.encode_plus(
                test_text,
                truncation=True,
                padding='max_length',
                max_length=300,
                return_tensors='pt'
            )

            output = model_BERT(**predict_input)[0]

            prediction_value = torch.argmax(output, axis=1).item()
            pred.append(prediction_value)
    elif test == 'syn':
        for test_text in test_texts_syn:
            predict_input = tokenizer_fine_tuned.encode_plus(
                test_text,
                truncation=True,
                padding='max_length',
                max_length=300,
                return_tensors='pt'
            )

            output = model_fine_tuned(**predict_input)[0]

            prediction_value = torch.argmax(output, axis=1).item()
            pred.append(prediction_value)
    elif test == 'mask':
        for test_text in test_texts_mask:
            predict_input = tokenizer_fine_tuned.encode_plus(
                test_text,
                truncation=True,
                padding='max_length',
                max_length=300,
                return_tensors='pt'
            )

            output = model_fine_tuned(**predict_input)[0]

            prediction_value = torch.argmax(output, axis=1).item()
            pred.append(prediction_value)
            
    total = len(pred)
    correct = 0
    for i in range(len(pred)):
        if pred[i] == test_labels[i]:
            correct += 1

    return (float(correct)/float(total))

# Results of stress testing

In [10]:
base_acc = compute_accuracy(test_labels)

print("Base accuracy: ", base_acc)

Base accuracy:  0.8628170894526035


In [11]:
syn_acc =  compute_accuracy(test_labels , test='syn')

print('Synonym Accuracy: ', syn_acc)

Synonym Accuracy:  0.8110814419225634


In [12]:
mask_acc =  compute_accuracy(test_labels, test='mask')

print('Mask Accuracy: ', mask_acc)

Mask Accuracy:  0.815086782376502
